In [4]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait


In [5]:
#Scraping NBC news wild fires with Selenium and Beautifulsoup



PATH = "./chromedriver_win32/chromedriver.exe"
driver = webdriver.Chrome(PATH)

link_list = []

driver.get("https://www.google.com/search?q=wildfire&rlz=1C1CHBF_frFR863FR863&biw=1920&bih=880&sxsrf=AOaemvI0XcPZB9YWw9GUVGwWTEXPDVqRxQ:1638967714934&source=lnms&tbm=nws&sa=X&ved=2ahUKEwjGlsnDntT0AhWTTcAKHeyuDk4Q_AUoAXoECAEQAw")

driver.find_element(By.XPATH, "//button[@class='VfPpkd-LgbsSe VfPpkd-LgbsSe-OWXEXe-k8QpJ VfPpkd-LgbsSe-OWXEXe-dgl2Hf nCP5yc AjY5Oe DuMIQc']").click() #accept google policy

for i in range(10):
    if i != 0:
        driver.find_element(By.ID, "pnnext").click()
    print(i)

    html_source = driver.page_source

    soup = BeautifulSoup(html_source, 'lxml')

    #Getting all g-card 
    g_card_list = soup.find_all("g-card")

    for g_card in g_card_list:
        a = g_card.find("a")
        link = a['href']
        link_list.append(link)

driver.quit()




C:\Users\SORLUC~1\AppData\Local\Temp/ipykernel_28448/2214517977.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


0
1
2
3
4
5
6
7
8
9


In [9]:
#Now that we have the link list, for each link try to scrape the article if there is one and the date if there is one.
my_timeout = 10

data = []

for i, link in enumerate(link_list):
    d = {}
    print(i+1, "/", len(link_list))

    title = soup.find('title')
    if title != None:
        d["Title"] = title.text


    d["Link"] = link

    try:
        html_text = requests.get(link, timeout=my_timeout).text
        soup = BeautifulSoup(html_text, 'lxml')

        time = soup.find('time')
        if time != None:
            if time.has_attr('datetime'):

                d["Date"] = time['datetime']
        
        article = soup.find('article')
        if article != None:
            paragraphs = article.find_all('p')
        big_p = ""
        for p in paragraphs:
            big_p = big_p + " " + p.text 
        
        if big_p != "":
            d["Content"] = big_p
    except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as err: #Requests takes way too long
        print('Server taking too long. Try again later for page number' + str(i))

    data.append(d)

gnews_df = pd.DataFrame(data)

0 / 100
1 / 100
2 / 100
3 / 100
4 / 100
5 / 100
6 / 100
7 / 100
8 / 100
9 / 100
10 / 100
11 / 100
12 / 100
13 / 100
14 / 100
15 / 100
16 / 100
17 / 100
18 / 100
19 / 100
20 / 100
21 / 100
22 / 100
23 / 100
24 / 100
25 / 100
26 / 100
27 / 100
28 / 100
29 / 100
30 / 100
31 / 100
32 / 100
33 / 100
34 / 100
35 / 100
36 / 100
37 / 100
38 / 100
39 / 100
40 / 100
Server taking too long. Try again later for page number40
41 / 100
42 / 100
43 / 100
44 / 100
45 / 100
46 / 100
47 / 100
48 / 100
49 / 100
50 / 100
51 / 100
52 / 100
53 / 100
54 / 100
55 / 100
56 / 100
57 / 100
58 / 100
59 / 100
60 / 100
61 / 100
62 / 100
63 / 100
64 / 100
65 / 100
66 / 100
67 / 100
68 / 100
69 / 100
70 / 100
71 / 100
72 / 100
73 / 100
74 / 100
75 / 100
76 / 100
Server taking too long. Try again later for page number76
77 / 100
78 / 100
79 / 100
80 / 100
81 / 100
82 / 100
83 / 100
84 / 100
85 / 100
86 / 100
87 / 100
88 / 100
89 / 100
90 / 100
91 / 100
92 / 100
93 / 100
94 / 100
95 / 100
96 / 100
97 / 100
98 / 100
99 

In [10]:
gnews_df

,Title,Link,Content,Date
0,Western Nebraska wildfire rages on,https://www.ijpr.org/wildfire/2021-12-08/after...,Lawmakers originally called for the hearing o...,NaN
1,"After long delay, California lawmakers hold wi...",https://www.cranbrooktownsman.com/news/indigen...,Lawmakers originally called for the hearing o...,NaN
2,403 Forbidden,https://www.theguardian.com/world/2021/dec/07/...,Climate crisis fanned intense blazes that emi...,NaN
3,"From Siberia to US west, wildfires spewed reco...",https://goldcountrymedia.com/news/213360/fores...,Climate crisis fanned intense blazes that emi...,NaN
4,\r\n\tForesthill boys place second at Wildfire...,https://www.foxcarolina.com/coronavirus/crews-...,Climate crisis fanned intense blazes that emi...,NaN
...,...,...,...,...
95,California Focus: Wildfire insurance crisis hi...,https://losangeles.cbslocal.com/2021/11/21/fir...,If Californians didn’t already know about the...,NaN
96,Firefighters Battling Wildfire In Riverside’s ...,https://www.nytimes.com/video/us/1000000080837...,If Californians didn’t already know about the...,NaN
97,Colorado Wildfire Prompts Evacuations - The Ne...,https://people.com/human-interest/3-bear-cubs-...,One cub's paws were so badly burned that it w...,NaN
98,3 Bear Cubs Recovering After Being Burned in W...,https://nmfireinfo.com/2021/11/09/wildfire-pre...,One cub's paws were so badly burned that it w...,NaN


In [11]:
gnews_df.isna().sum()

Title       1
Link        0
Content     7
Date       75
dtype: int64